# How to use the benchmark
There are 5 different test splits in the benchmark. A model should be tested on each split to evaluate its performances accurately. The means that the model must be trained five times on different training and validation sets that are separated from the current test split. The hyperparameters of the model should stay the same for all splits. This notebook will show a simple example on how to use the benchmark.

In [1]:
# Utility function for our model in the example. This is not part of QMAP.
import torch
from transformers import EsmTokenizer, EsmModel
import numpy as np
from typing import List
from tqdm import tqdm

def generate_esm2_embeddings(protein_sequences: List[str],
                             model_name: str = "facebook/esm2_t33_650M_UR50D",
                             device: str = None,
                             batch_size: int = 64) -> np.ndarray:
    """
    Generate protein embeddings using ESM2 model with mean pooling.

    Args:
        protein_sequences (List[str]): List of protein sequences (amino acid sequences)
        model_name (str): Name of the ESM2 model to use. Options include:
            - "facebook/esm2_t6_8M_UR50D" (8M parameters)
            - "facebook/esm2_t12_35M_UR50D" (35M parameters)
            - "facebook/esm2_t30_150M_UR50D" (150M parameters)
            - "facebook/esm2_t33_650M_UR50D" (650M parameters) - default
            - "facebook/esm2_t36_3B_UR50D" (3B parameters)
            - "facebook/esm2_t48_15B_UR50D" (15B parameters)
        device (str): Device to run the model on ('cuda', 'cpu', or None for auto-detection)
        batch_size (int): Batch size for processing sequences

    Returns:
        np.ndarray: Array of shape (num_sequences, embedding_dim) containing embeddings
                   in the same order as input sequences
    """

    # Set device
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Load tokenizer and model
    tokenizer = EsmTokenizer.from_pretrained(model_name)
    model = EsmModel.from_pretrained(model_name)
    model.to(device)
    model.eval()

    all_embeddings = []

    # Process sequences in batches
    for i in tqdm(range(0, len(protein_sequences), batch_size)):
        batch_sequences = protein_sequences[i:i + batch_size]

        # Tokenize sequences
        inputs = tokenizer(batch_sequences,
                           return_tensors="pt",
                           padding=True,
                           truncation=True,
                           max_length=1024)

        # Move to device
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Generate embeddings
        with torch.no_grad():
            outputs = model(**inputs)
            hidden_states = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)

        # Process each sequence in the batch
        for j in range(len(batch_sequences)):
            # Get attention mask for this sequence (to exclude padding tokens)
            attention_mask = inputs['attention_mask'][j]
            seq_embeddings = hidden_states[j][attention_mask.bool()]  # Remove padding

            # Remove special tokens (CLS and EOS tokens)
            seq_embeddings_no_special = seq_embeddings[1:-1]  # Remove first (CLS) and last (EOS) tokens

            # Mean pooling
            seq_embedding = seq_embeddings_no_special.mean(dim=0)
            all_embeddings.append(seq_embedding.cpu().numpy())

    return np.array(all_embeddings)

/Users/anthonylavertu/mac_docs/pycharmProjects/QMAP/QMAP/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/anthonylavertu/mac_docs/pycharmProjects/QMAP/QMAP/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## MIC

In [8]:
from qmap.benchmark import QMAPBenchmark, DBAASPDataset

# Imports for the example
from sklearn.linear_model import LinearRegression
import numpy as np

# Step 1: Load the training dataset.
# For this example, we will load the DBAASP dataset that is packaged with QMAP. We want to keep only samples that have a target for Escherichia coli.
dataset = (DBAASPDataset()
           .with_bacterial_targets(['Escherichia coli'])
           )


# Loop over all splits of the benchmark
for split in range(5):
    # Step 2: Load the benchmark.
    # For this example, we will only consider the specie Escherichia coli. So we filter out any sample that does not have a target for this specie.
    benchmark = QMAPBenchmark(split).with_bacterial_targets(['Escherichia coli'])
    # Step 3: Get the training mask. This means the sequences that can be used to train the model because they are dissimilar enough from the test set.
    train_mask = benchmark.get_train_mask([seq for seq in dataset.sequences])
    # Step 4: Filter the dataset to only keep the sequences that can be used for training
    train_dataset = dataset[train_mask]

    # Step 5: Train the model on the training dataset
    # For this example, we will use a linear model on top of the encoder's embeddings
    y_train = np.log10(np.array([sample.targets['Escherichia coli'].consensus for sample in train_dataset]))
    X_train = generate_esm2_embeddings(train_dataset.sequences)
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Step 6: Get the test sequences and their targets
    # The only inputs we have are the sequences because we turned off all options. However, if you want to include C and N terminal modifications, you can do this in the benchmark constructor. When accessing the inputs attribute, you will receive the sequences, the C terminal modifications, and the N terminal modifications. The same concept applies for unusual amino acids or species as inputs which will include the specie name with the sequence.
    test_sequences = benchmark.sequences

    # Step 7: Get the predictions of the model on the test set
    X_test = generate_esm2_embeddings(test_sequences)
    y_pred = model.predict(X_test)
    print(y_pred.shape)

    # Conver the predictions to a format compatible with the benchmark evaluation (List of dictionaries, where each key is a specie name or HC50)
    y_pred = [{'Escherichia coli': pred} for pred in y_pred]

    # Step 8: Evaluate the model
    # The benchmark provides a method to evaluate the model on the test set given the predictions.
    results = benchmark.compute_metrics(y_pred)['Escherichia coli']

    # You can also get the results as a dictionary
    results_dict = results.dict()

Loaded binary mask from cache: /Users/anthonylavertu/Library/Caches/pwiden_engine/binary_mask_e896fd46047b26e442355e893b5e544d36f44d7e940d412893cc4b8ac9c6945d_thresh_0.6000.npy
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 134/134 [08:27<00:00,  3.79s/it]
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 33/33 [03:00<00:00,  5.46s/it]
/Users/anthonylavertu/mac_docs/pycharmProjects/QMAP/QMAP/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by

(2112,)
QMAPMetrics(property: Escherichia coli; 2112):
 - RMSE: 0.7276
 - MSE: 0.5295
 - MAE: 0.5682
 - R2: 0.0711
 - Spearman: 0.4281
 - Kendall's Tau: 0.2973
 - Pearson: 0.4563


Saved binary mask to cache: /Users/anthonylavertu/Library/Caches/pwiden_engine/binary_mask_887b7ab0e9a9db89de59c11616da9d7ac7e3e8f2161a20dee8f77f9a2b47cf9f_thresh_0.6000.npy
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 4/130 [00:14<07:36,  3.63s/it]


KeyboardInterrupt: 

## Subsets
The benchmark can be filtered to evaluate the model on different environment conditions. This is done easily using the filters methods of the benchmark. There are multiple filter methods pre-implemented that starts with the `with_` prefix.

For example, we can obtain the high-efficiency subset used in the paper as follows:

In [9]:
# This gets all samples that have an efficiency below 10 uM for any bacterias in its targets.
high_eff = benchmark.with_efficiency_below(10.)

# The high_eff object is of the same type as benchmark, so you can use all its methods to evaluate the model on this subset.
# results_high_eff = high_eff.compute_metrics(...)
print(high_eff)

DBAASP ID  Sequence                                 N Terminal   C Terminal   # Targets  HC50  
3681       GLIDTIKNMALNAAKSAGVSVLNTLSCKLSKTC        Free         Free         3          nan
7811       RFRRLRCKTRCRLKKI                         Free         AMD          5          nan
6813       GIHKILKYGKPS                             Free         AMD          5          nan
5744       NVTPATKPTPSKPGYCRVMDELILCPDPPLSKDLCKNDSD Free         Free         3          nan
                                               ...
7641       RPPQFTRAQWFAIQHISLNPPRSTIAMRAINNYRWRSKNQ Free         Free         6          9.50
9073       fwGalakGalklipslfssfskkd                 Free         Free         4          10.00
14831      FYPRPYRPPYLPDPRPFPRPLPAFGHEFRRH          Free         Free         3          nan
2144       FFGHLFKLATKIIPSLFQ                       Free         Free         7          nan

Total of 1508 samples


## Extending filters
You can also create your own filter function by making a callback function that takes as input a `Sample` and returns a boolean indicating whether to keep the sample or not (True to keep). You can then use the `filter` method of the benchmark to create a new benchmark with only the samples that satisfy the condition.

For example, here we create a filter to keep only samples that have a hemolytic activity (HC50) above 100 uM.

In [14]:
from qmap.benchmark import Sample
import math
def low_hemo(sample: Sample) -> bool:
    """Keep only samples that have a hemolytic activity (HC50) above 100 uM."""
    hc50 = sample.hc50.consensus
    if math.isnan(hc50):
        return False
    return hc50 > 100.

no_hemo_benchmark = benchmark.filter(low_hemo)
print(no_hemo_benchmark)

DBAASP ID  Sequence                                 N Terminal   C Terminal   # Targets  HC50  
8864       GILGKLWEGVKSIF                           Free         AMD          13         148.00
22787      AVLIPFKVKFGKVKFGKVKFRCKAAFC              Free         Free         7          128.00
1970       GAFGDLLKGVAKEAGMKLLNMAQCKLSGKC           Free         Free         8          520.00
4588       LLIILRRRWRKQARARSK                       Free         AMD          6          200.00
                                               ...
14228      ILKKIWEGIKSLF                            Free         AMD          10         136.00
3269       GMWSKILGKLIR                             Free         AMD          4          143.00
2284       GLKEVLHSTKKFAKGFITGLTGQ                  Free         Free         2          160.00
3275       GKWSKILGKLIR                             Free         AMD          4          200.00

Total of 216 samples


# Hemolytic activity
Similarly, you can evaluate the model on hemolytic activity by using the 'hc50' key in `compute_metrics`. You can subsample the benchmark to only keep samples that have a hemolytic activity target using the `with_hc50_targets` method.